In [1]:
from psaw import PushshiftAPI
import pandas as pd
from datetime import date, timedelta,datetime as dt
import concurrent.futures

C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import os
import os.path as osp
from pathlib import Path

In [3]:
#manually downloaded data from kaggle
#df_main = pd.read_csv('news.csv')
#df_main.sort('date')
data_path = osp.join(Path(os.getcwd()).parent,'Data')

In [4]:
api = PushshiftAPI()

In [5]:
def fetch_reddit_data(time_frame):
    topics_lst = []
    
    after_epoch  =   int(time_frame[0].timestamp())
    before_epoch =   int(time_frame[1].timestamp()) 
    
    submision_date = time_frame[0].strftime('%d/%m/%Y')
    
    push_lst = list(api.search_submissions(after=after_epoch,
                            before = before_epoch,
                            subreddit='worldnews',
                            filter=['score', 'title'],
                            limit=35))
    
    for submission in push_lst:
        topic_dict = {}
       
        topic_dict["date"]=submision_date
        topic_dict["title"]= submission.title
        topic_dict["score"]= submission.score
        topics_lst.append(topic_dict)
        
    return topics_lst

In [6]:
def download_data(sdate,edate):
    timeframe_lst = []
    delta = edate - sdate       # as timedelta
    for i in range(delta.days + 1):
        curr_day = sdate + timedelta(days=i)
        next_day = sdate + timedelta(days=i + 1)
        timeframe_lst.append((curr_day,next_day))
    news_lst = []
    with concurrent.futures.ThreadPoolExecutor(max_workers = 50 ) as executor:
        results = executor.map(fetch_reddit_data,timeframe_lst)
        for res in results:                
            news_lst.extend(res)
            
    print('news list lenght ' ,len(news_lst))
            
    df =  pd.DataFrame(news_lst)
    df['Date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by=['Date','score'],ascending=[True, False])  
    
    return df

In [ ]:
sdate = dt(2019, 1, 1)   # start date
edate = dt(2019, 12, 31) 
df_2019 = download_data(sdate,edate)
df_2019.to_csv(data_path + '/reddit-yearly-download/news_2019.csv')

In [ ]:
sdate = dt(2018, 1, 1)   # start date
edate = dt(2018, 12, 31) 
df_2018 = download_data(sdate,edate)
df_2018.to_csv(data_path + '/reddit-yearly-download/news_2018.csv')


In [ ]:
sdate = dt(2017, 1, 1)   # start date
edate = dt(2017, 12, 31) 
df_2017 = download_data(sdate,edate)
df_2017.to_csv(data_path + '/reddit-yearly-download/news_2017.csv')

In [ ]:
sdate = dt(2016, 7, 2)   # start date
edate = dt(2016, 12, 31) 
df_2016 = download_data(sdate,edate)
df_2016.to_csv(data_path + '/reddit-yearly-download/news_2016.csv')

In [ ]:
sdate = dt(2020, 1, 1)   # start date
edate = dt(2020, 9, 1) 
df_2020 = download_data(sdate,edate)
df_2020.to_csv(data_path + '/reddit-yearly-download/news_2020.csv')
